# Segment Tree
> 동적으로 변하는 배열의 구간(segment) 정보에 대한 질문에 효율적으로 답하기 위한 자료구조. "3번째 부터 11번째 원소중 최솟값은 무엇인가?" 와 같은 질문을 빠르게 처리할 수 있다.

일반적인 방법으로는 매번 해당 구간을 전부 순회해야 하지만, 세그먼트 트리를 사용하면 `O(logN)` 으로 해결할 수 있다.

## 핵심 아이디어 : 분할 정복
세그먼트 트리의 핵심은 "큰 구간을 작은 구간들로 나우어 미리 계산해두자" 라는 것이다. 이렇게 하여 나중에 질문이 들어왔을 때 이미 계산된 작은 구간들을 조합해서 빠르게 답할 수 있다.


```text
===================================================================================
                    세그먼트 트리 구조 (최솟값/최댓값)
===================================================================================

원본 배열:
인덱스:  1    2    3    4    5    6    7    8    9   10
값:     75   30  100   38   50   51   52   20   81    5

===================================================================================

트리 구조 (각 노드는 [구간, min, max] 형태):

                            [1-10, 5, 100]
                                 (노드 1)
                                    |
                    +---------------+---------------+
                    |                               |
              [1-5, 30, 100]                  [6-10, 5, 81]
                 (노드 2)                        (노드 3)
                    |                               |
          +---------+---------+           +---------+---------+
          |                   |           |                   |
    [1-3, 30, 100]      [4-5, 38, 50]  [6-8, 20, 52]   [9-10, 5, 81]
       (노드 4)           (노드 5)        (노드 6)         (노드 7)
          |                   |              |                 |
      +---+---+           +---+---+      +---+---+        +---+---+
      |       |           |       |      |       |        |       |
  [1-2,    [3-3,      [4-4,   [5-5,  [6-7,    [8-8,   [9-9,   [10-10,
  30,75]   100,100]   38,38]  50,50] 51,52]   20,20]  81,81]  5,5]
  (노드8)   (노드9)    (노드10) (노드11) (노드12) (노드13) (노드14) (노드15)
     |
  +--+--+
  |     |
[1-1, [2-2,
75,75] 30,30]
(노드16)(노드17)

===================================================================================

노드 번호와 인덱스 관계:
- 부모 노드: k
- 왼쪽 자식: k * 2
- 오른쪽 자식: k * 2 + 1

예시:
- 노드 1의 왼쪽 자식: 1 * 2 = 2
- 노드 1의 오른쪽 자식: 1 * 2 + 1 = 3
- 노드 2의 왼쪽 자식: 2 * 2 = 4
- 노드 2의 오른쪽 자식: 2 * 2 + 1 = 5

===================================================================================
```



## 1. 트리 초기화
재귀적으로 분할하며 초기화를 진행한다. 큰 구간을 절반씩 나누면서 리프노드까지 내려간 다음, 다시 올라오면서 각 구간의 정보를 계산한다.

In [ ]:
fun init(node: Int, start: Int, end: Int): LongArray {
    // 리프 노드: 구간에 원소가 하나만 있는 경우
    if (start == end) {
        return longArrayOf(arr[start], arr[start]).also { tree[node] = it }
    }

    // 구간을 절반으로 나눔
    val mid = start + (end - start) / 2


    // 왼쪽 절반 초기화
    val left = init(node * 2, start, mid)


    // 오른쪽 절반 초기화
    val right = init(node * 2 + 1, mid + 1, end)

    // 왼쪽과 오른쪽의 결과를 합침
    val min = min(left[0], right[0])
    val max = max(left[1], right[1])

    return longArrayOf(min, max).also { tree[node] = it }
}

## 2. 쿼리
검색하고자 하는 구간의 정보를 쿼리하기 위해서는 질의 구간과 현재 구간이 겹치는지 확인하여 트리를 탐색하여야 한다.

In [ ]:
fun query(node: Int, start: Int, end: Int, left: Int, right: Int): LongArray {
    // 경우 1: 현재 구간이 질의 구간과 전혀 겹치지 않음
    if (right < start || end < left) {
        return longArrayOf(Long.Companion.MAX_VALUE, Long.Companion.MIN_VALUE)
    }

    // 경우 2: 현재 구간이 질의 구간에 완전히 포함됨
    if (left <= start && end <= right) {
        return tree[node]
    }

    // 경우 3: 현재 구간이 질의 구간과 일부만 겹침
    // 왼쪽과 오른쪽 자식을 모두 탐색해야 함
    val mid = start + (end - start) / 2

    val node_left = query(node * 2, start, mid, left, right)
    val node_right = query(node * 2 + 1, mid + 1, end, left, right)

    val min = min(node_left[0], node_right[0])
    val max = max(node_left[1], node_right[1])

    return longArrayOf(min, max)
}

```
쿼리 실행 과정: query(1, 1, 10, 3, 5)
질문: "3번째부터 5번째 원소의 최솟값과 최댓값은?"
배열: [75, 30, 100, 38, 50, 51, 52, 20, 81, 5]
===================================================================================

query(1, 1, 10, 3, 5) 호출
  노드 1: 구간 [1-10], 질의 [3-5]
  검사: 3 <= 1? NO, 10 <= 5? NO
  => 일부 겹침! 자식 노드 탐색 필요
  mid = 5
  |
  +-- query(2, 1, 5, 3, 5) 호출 [왼쪽 자식]
  |     노드 2: 구간 [1-5], 질의 [3-5]
  |     검사: 3 <= 1? NO, 5 <= 5? YES
  |     => 일부 겹침! 자식 노드 탐색 필요
  |     mid = 3
  |     |
  |     +-- query(4, 1, 3, 3, 5) 호출
  |     |     노드 4: 구간 [1-3], 질의 [3-5]
  |     |     검사: 5 < 1? NO, 3 < 3? NO
  |     |     검사: 3 <= 1? NO, 3 <= 5? YES
  |     |     => 일부 겹침! 자식 노드 탐색 필요
  |     |     mid = 2
  |     |     |
  |     |     +-- query(8, 1, 2, 3, 5) 호출
  |     |     |     노드 8: 구간 [1-2], 질의 [3-5]
  |     |     |     검사: 5 < 1? NO, 2 < 3? YES
  |     |     |     => 겹치지 않음!
  |     |     |     반환: [MAX_VALUE, MIN_VALUE]
  |     |     |
  |     |     +-- query(9, 3, 3, 3, 5) 호출
  |     |     |     노드 9: 구간 [3-3], 질의 [3-5]
  |     |     |     검사: 5 < 3? NO, 3 < 3? NO
  |     |     |     검사: 3 <= 3? YES, 3 <= 5? YES
  |     |     |     => 완전히 포함됨! ✓
  |     |     |     반환: tree[9] = [100, 100]
  |     |     |
  |     |     └─> 병합: min(MAX_VALUE, 100) = 100
  |     |               max(MIN_VALUE, 100) = 100
  |     |         반환: [100, 100]
  |     |
  |     +-- query(5, 4, 5, 3, 5) 호출
  |     |     노드 5: 구간 [4-5], 질의 [3-5]
  |     |     검사: 5 < 4? NO, 5 < 3? NO
  |     |     검사: 3 <= 4? YES, 5 <= 5? YES
  |     |     => 완전히 포함됨! ✓
  |     |     반환: tree[5] = [38, 50]
  |     |
  |     └─> 병합: min(100, 38) = 38
  |               max(100, 50) = 100
  |         반환: [38, 100]
  |
  +-- query(3, 6, 10, 3, 5) 호출 [오른쪽 자식]
  |     노드 3: 구간 [6-10], 질의 [3-5]
  |     검사: 5 < 6? YES
  |     => 겹치지 않음!
  |     반환: [MAX_VALUE, MIN_VALUE]
  |
  └─> 병합: min(38, MAX_VALUE) = 38
            max(100, MIN_VALUE) = 100
      최종 반환: [38, 100]

===================================================================================

결과 해석:
구간 [3, 5]의 원소들: 100, 38, 50
최솟값: 38 ✓
최댓값: 100 ✓

===================================================================================

시각적 표현:

                    [1-10]
                    (일부 겹침)
                      /    \
                     /      \
              [1-5]           [6-10]
           (일부 겹침)        (겹치지 않음) ✗
              /    \
             /      \
        [1-3]        [4-5]
     (일부 겹침)    (완전 포함) ✓
       /    \
      /      \
   [1-2]     [3-3]
(겹치지 않음)✗ (완전 포함) ✓

사용된 노드: 노드 9 (구간 [3-3]) + 노드 5 (구간 [4-5])
=> 딱 2개의 노드만으로 답을 구할 수 있다!!
```

### 3. 항등원
항등원이란, 다른 값과 연산했을 때 그 값을 변화시키지 않는 특별한 값을 일컫는다. 세그먼트 트리에서는 조회 과정에서 질의구간과 완전히 겹치지 않는 구간을 만났을 때, 그 반환결과가 결과에 영향을 주지 않도록 하기 위해 사용한다.

예시:
- 덧셈의 항등원: 0          -> a + 0 = a
- 곱셈의 항등원: 1          -> a × 1 = a
- 최솟값의 항등원: 무한대     -> min(a, ∞) = a
- 최댓값의 항등원: 음의 무한대 -> max(a, -∞) = a

```
===================================================================================

세그먼트 트리에서의 사용:

예시 코드에서는 [최솟값, 최댓값]을 함께 저장한다.
따라서 항등원은 [Long.MAX_VALUE, Long.MIN_VALUE] 이다. (수학적으로는 양/음의 무한대 이지만 컴퓨터가 연산에 사용할 수 있어야 하므로 Long 볌위에서 사용)

왜 이렇게 사용할까?

예를 들어, 두 구간의 결과를 병합한다고 생각해보자:
- 구간 A: [38, 50] (실제 데이터)
- 구간 B: [MAX_VALUE, MIN_VALUE] (겹치지 않는 구간)

병합 과정:
min_result = min(38, MAX_VALUE) = 38 ✓
max_result = max(50, MIN_VALUE) = 50 ✓

결과: [38, 50] - 구간 B가 결과에 영향을 주지 않는다!

===================================================================================

실제 쿼리 예제로 확인:

query(1, 1, 10, 3, 5)를 실행할 때:

1단계: query(2, 1, 5, 3, 5) → [38, 100]
2단계: query(3, 6, 10, 3, 5) → [MAX_VALUE, MIN_VALUE] (겹치지 않음)

병합:
min(38, MAX_VALUE) = 38
max(100, MIN_VALUE) = 100
결과: [38, 100] ✓

만약 항등원을 잘못 설정했다면?
예) [0, 0]을 반환한다면:
min(38, 0) = 0 ✗ (잘못된 결과!)
max(100, 0) = 100 ✓ (우연히 맞음)

예) [-1, -1]을 반환한다면:
min(38, -1) = -1 ✗ (잘못된 결과!)
max(100, -1) = 100 ✓ (우연히 맞음)

===================================================================================

다른 연산의 항등원:

연산 종류              | 항등원
--------------------|--------------------
구간 합 (Sum)        | 0
구간 곱 (Product)    | 1
최솟값 (Minimum)     | Long.MAX_VALUE (∞)
최댓값 (Maximum)     | Long.MIN_VALUE (-∞)
최대공약수 (GCD)      | 0
최소공배수 (LCM)      | 1
비트 OR             | 0
비트 AND            | 모든 비트가 1인 값
비트 XOR            | 0

===================================================================================

핵심 원리:
항등원은 "아무 영향도 주지 않는 값"이다.
세그먼트 트리에서 겹치지 않는 구간을 만날 때,
그 구간이 최종 결과에 영향을 주면 안 되므로 항등원을 반환한다.
```

## 4. update
만약 배열이 변하지 않는다면 다른 방법들도 효과적일 수 있다. 특히 구간의 합을 구하기만 한다면 누적합과 같은 기법이 더 간단하고 빠를 수 있다. 다만 세그먼트 트리는 정보를 집계하고자 하는 대상 배열이 동적으로 변화하는 과정에서 더욱 효과적이다.

온라인 게임 서버에서 플레이어들의 점수를 관리한다고 가정한다면?
- 플레이어가 퀘스트를 완료하면 점수가 올라간다 (대상 배열 값 변경)
- 관리자는 특정 레벨 구간의 최고 점수를 실시간으로 조회한다 (구간 쿼리)
- 이런 작업이 초당 수천 번 일어난다!

또 다른 예시로 주식 거래 시스템을 생각해보자.
- 주식 가격이 실시간으로 변한다 (대상 배열 값 변경)
- 특정 시간대의 최고가, 최저가를 조회한다 (시계열 구간 쿼리)
- 이것도 밀리초 단위로 계속 일어난다!

이런 상황에서 세그먼트 트리가 최고의 선택이 될 수 있다.


### 세그먼트 트리에서 값을 업데이트하는 방법

기본 아이디어:
1. 리프 노드(실제 배열 값)를 찾아서 변경하기
2. 그 리프 노드에서 루트까지 올라가면서 영향받는 모든 구간을 업데이트
3. 트리의 높이가 log N이므로, log N개의 노드만 업데이트!!


In [ ]:
fun update(node: Int, start: Int, end: Int, index: Int, newValue: Int) {
    // 경우 1: 현재 구간이 변경할 인덱스를 포함하지 않음
    if (index < start || end < index) {
        return
    }
    // 경우 2: 리프 노드에 도달 (변경할 위치를 찾음!)
    if (start == end) {
        arr[index] = newValue.toLong()
        tree[node] = newValue.toLong()
        return
    }

    // 경우 3: 내부 노드 - 자식 노드들을 업데이트하고 병합
    val mid = start + (end - start) / 2
    update(node * 2, start, mid, index, newValue)
    update(node * 2 + 1, mid + 1, end, index, newValue)
    // 자식 노드들이 업데이트되었으므로, 현재 노드도 다시 계산
    tree[node] = tree[node * 2] + tree[node * 2 + 1] // 정보를 갱신하는 과정. 구간합이 아니라 구간평균이라면 두 자식 노드의 평균을 저장하여야 합다.
}

```text
===================================================================================

다른 방법들과의 비교
===================================================================================

방법 1: 누적합 (Prefix Sum)
─────────────────────────────

누적합은 구간 합을 구할 때 매우 효율적이다.

원본 배열: [3, 5, 2, 7, 1, 4]
누적합 배열: [3, 8, 10, 17, 18, 22]

구간 [2, 4]의 합 = 누적합[4] - 누적합[1] = 17 - 3 = 14
(원소: 5 + 2 + 7 = 14) ✓

성능:
- 초기화: O(N)
- 구간 합 쿼리: O(1) ← 매우 빠름!
- 값 업데이트: O(N) ← 치명적인 단점!

왜 업데이트가 O(N)일까?
만약 arr[2] = 5를 arr[2] = 10으로 바꾼다면?
누적합 배열의 2번 이후 모든 값을 다시 계산해야 합니다!
- 기존: [3, 8, 10, 17, 18, 22]
- 변경: [3, 8, 15, 22, 23, 27] ← 4개를 다시 계산

결론: 배열이 자주 변하면 누적합은 비효율적임

─────────────────────────────

방법 2: 매번 순회
─────────────────────────────

가장 단순한 방법이다.

성능:
- 초기화: O(1)
- 구간 쿼리: O(N)
- 값 업데이트: O(1) ← 빠름!

하지만 쿼리가 많으면 전체적으로 매우 매우 느리다.

─────────────────────────────

방법 3: 세그먼트 트리
─────────────────────────────

균형잡힌 성능을 제공한다.

성능:
- 초기화: O(N)
- 구간 쿼리: O(log N) ← 빠름!
- 값 업데이트: O(log N) ← 빠름!

쿼리도 빠르고, 업데이트도 빠르다!
이것이 세그먼트 트리가 동적 배열에 최적인 이유.

===================================================================================

비교 표: 10만 개 원소, 10만 번의 쿼리와 업데이트
===================================================================================

작업 시나리오: 쿼리 5만 번 + 업데이트 5만 번

┌─────────────┬──────────┬─────────────┬──────────────┬─────────┐
│   방법      │  초기화  │  쿼리 1회   │ 업데이트 1회 │  총합   │
├─────────────┼──────────┼─────────────┼──────────────┼─────────┤
│ 매번 순회   │   O(1)   │   O(N)      │    O(1)      │  50억   │
│             │   즉시   │  100,000    │     1        │  연산   │
│             │          │             │              │ (느림!) │
├─────────────┼──────────┼─────────────┼──────────────┼─────────┤
│ 누적합      │  O(N)    │   O(1)      │    O(N)      │  50억   │
│             │ 100,000  │     1       │  100,000     │  연산   │
│             │          │             │              │ (느림!) │
├─────────────┼──────────┼─────────────┼──────────────┼─────────┤
│세그먼트트리 │  O(N)    │  O(log N)   │  O(log N)    │ 180만   │
│             │ 100,000  │    17       │     17       │  연산   │
│             │          │             │              │ (빠름!) │
└─────────────┴──────────┴─────────────┴──────────────┴─────────┘

세상에나, 세그먼트 트리가 약 2,700배 빠르다!



```text

===================================================================================

update 실행 과정 시각화
===================================================================================

예제: 5번 인덱스의 값을 50 → 99로 변경

원본 배열:
인덱스:  1    2    3    4    5    6    7    8    9   10
값:     75   30  100   38   50   51   52   20   81    5
                           ↓↓
변경 후: 75   30  100   38   99   51   52   20   81    5

트리 구조에서 변경 경로:

                          [1-10]                    ← 업데이트 4
                         min=5→5
                         max=100→100
                            |
              ┌─────────────┴─────────────┐
              |                           |
          [1-5]                        [6-10]
         min=30→30                    (변화 없음)
         max=100→100    ← 업데이트 3
              |
      ┌───────┴───────┐
      |               |
   [1-3]           [4-5]              ← 업데이트 2
 (변화 없음)      min=38→38
                 max=50→99
                     |
              ┌──────┴──────┐
              |             |
           [4-4]         [5-5]        ← 업데이트 1 (리프 노드)
         (변화 없음)    min=50→99
                       max=50→99

업데이트 순서:
1. 리프 노드 [5-5] 찾기 (재귀로 내려감)
2. arr[5] = 99, tree[노드] = [99, 99]
3. 위로 올라가면서 부모 노드들 갱신:
   - [4-5]: min(38, 99) = 38, max(38, 99) = 99
   - [1-5]: min(30, 38) = 30, max(100, 99) = 100
   - [1-10]: min(30, 5) = 5, max(100, 81) = 100

업데이트된 노드 개수: 4개 (트리의 높이만큼)
시간 복잡도: O(log N) ✓

핵심 관찰:
- 전체 트리를 다시 만들 필요가 없다!
- 영향받는 경로(루트까지의 경로)만 업데이트하면 된다.
- 경로의 길이는 트리의 높이 = log N
```

```text
===================================================================================

실전 예제: 동적 배열 관리
===================================================================================

시나리오: 실시간 주식 가격 모니터링 시스템

요구사항:
- 100개 종목의 가격을 추적
- 가격은 초당 수십 번 변경됨
- 특정 범위 종목의 최고가/최저가를 실시간 조회
- 1초에 수백 번의 쿼리 발생

누적합으로 구현했다면?
- 가격 변경마다 O(N) = 100번의 연산 필요
- 초당 수십 번 변경 = 수천 번의 연산
- 서버가 감당하기 어려움!

세그먼트 트리로 구현하면?
- 가격 변경마다 O(log N) = 7번의 연산
- 초당 수십 번 변경 = 수백 번의 연산
- 여유롭게 처리 가능!
```


In [ ]:
// 실시간 주식 가격 모니터링 - 예시 코드
object StockPriceMonitor {
    var N: Int = 100 // 100개 종목
    var prices: IntArray
    var tree: Array<LongArray?>?

    @JvmStatic
    fun main(args: Array<String>) {
        prices = IntArray(N + 1)
        tree = arrayOfNulls<LongArray>(N * 4)

        // 초기 가격으로 트리 구축
        for (i in 1..N) {
            prices[i] = getInitialPrice(i)
        }
        init(1, 1, N)

        // 실시간 업데이트 및 쿼리
        while (true) {
            // 시나리오 1: 종목 45의 가격이 변경됨
            val stockId = 45
            val newPrice: Int = receiveNewPrice(stockId)
            update(1, 1, N, stockId, newPrice)

            // O(log N) = O(7) 정도의 연산만으로 즉시 반영!

            // 시나리오 2: 종목 20~60 범위의 최고가 조회
            val result: LongArray = query(1, 1, N, 20, 60)
            println("최저가: " + result[0])
            println("최고가: " + result[1])

            // O(log N) = O(7) 정도의 연산으로 즉시 조회!

            // 시나리오 3: 여러 종목 동시 변경
            update(1, 1, N, 23, 15000)
            update(1, 1, N, 67, 32000)
            update(1, 1, N, 89, 8500)
            // 각각 O(log N)으로 빠르게 처리!
        }
    }
}

### 언제 세그먼트 트리를 사용해야 할까?

```text
세그먼트 트리를 사용해야 하는 경우:
────────────────────────────────────
✓ 배열의 값이 자주 변경될 때
✓ 구간 쿼리가 자주 발생할 때
✓ 업데이트와 쿼리가 섞여서 일어날 때
✓ 실시간 데이터 처리가 필요할 때
✓ O(log N)의 일관된 성능이 필요할 때

예시:
- 온라인 게임 점수 관리
- 실시간 주식 시세 분석
- 네트워크 트래픽 모니터링
- 센서 데이터 실시간 분석
- 데이터베이스 인덱싱

다른 방법을 고려해볼 수 있는 경우:
────────────────────────────────────
○ 배열이 전혀 변하지 않을 때 → 누적합, 희소 테이블(Sparse Table)
○ 업데이트만 많고 쿼리가 거의 없을 때 → 일반 배열
○ 쿼리만 많고 업데이트가 거의 없을 때 → 누적합
○ 배열이 매우 작을 때 (N < 100) → 그냥 순회해도 충분

```


### update 함수의 핵심 원리
세그먼트 트리의 업데이트가 빠른 이유는 다음과 같다:

1. 국소적 영향 (Localized Impact)
   - 하나의 값이 변경되어도 전체 트리가 아닌 루트까지의 경로에 있는 노드들만 영향을 받는다.
   - 이는 트리의 높이인 O(log N)개의 노드임.

2. 상향식 전파 (Bottom-up Propagation)
   - 리프 노드에서 시작해서 위로 올라가면서 업데이트
   - 각 레벨에서 자식들의 값을 병합하기만 하면 됨
   - 간단하고 효율적인 과정

3. 구조적 이점 (Structural Advantage)
   - 완전 이진 트리 구조
   - 배열로 구현되어 캐시 친화적
   - 인덱스 계산이 간단 (node * 2, node * 2 + 1)
